# Clubhouse Room Analytics

In [1]:
# import datetime # haven't used module yet
# import numpy as np # did I use this module?
import pandas as pd
# from matplotlib import pyplot as plt
# import seaborn as sns


pd.set_option('display.max_columns', None,
              'display.max_rows', None,
              'display.float_format', '{:,.2f}'.format,
              'display.precision', 3
             )


# %matplotlib inline
# plt.style.use('fivethirtyeight')
# plt.rcParams['figure.figsize'] = (12, 10)


# This one is optional -- change graphs to SVG
# Only use if you don't have a lot of points/lines
# on your graph.
# Can also just use ['retina'] if you don't want SVG.
# %config InlineBackend.figure_formats = ['retina', 'svg']

# sns.set_style('whitegrid')

In [2]:
df = pd.read_excel('ch_data.xlsx')

In [3]:
# df.sample(5)

In [4]:
# df.info()

In [5]:
# list(df.columns.sort_values())

In [6]:
df.shape
df = df.drop_duplicates(subset='Room ID', keep='first', ignore_index=True)
df.shape

(803, 47)

(692, 47)

In [7]:
# columns to be removed
columns_drop = ['xEvent', 'xAvg Listener Time 2 [ClubHub]', 'xAvg Listener Time [ClubHub]', 
                'xAvg Speaker Time 2 [ClubHub]', 'xAvg Speaker Time [ClubHub]', 'xCalc', 
                'xDuration 2 [ClubHub]', 'xDuration [CH Tools]', 'xDuration [ClubHub]', 'xTime']

# define column dtypes
columns_date = ['Event']

columns_cat = ['Club', 'Tag 1', 'Tag 2']

columns_url = ['URL [CH Tools]','URL [ClubHub]']

columns_pct = ['Bounce [ClubHub]', 'Joined [In Last 7 Days] [ClubHub]', 'Joined [In Last Month] [ClubHub]',
               'Joined [Over A Month] [ClubHub]', ]

columns_float = ['Avg Listened Minutes [New Members] [CH Tools]', 'Avg Listened Minutes [Old Members] [CH Tools]',
                'Duration [CH Tools]', 'Duration [ClubHub]',]

# ...and because OCD
# don't like my integers to not display trailing zeros
columns_int = ['Avg Listener Minutes [ClubHub]', 'Avg Speaker Minutes [ClubHub]', 'Avg User Time [CH Tools]',
               'Followers [CH Tools]', 'Listeners [CH Tools]', 'Listeners [New Members] [CH Tools]',
               'Listeners [Old Members] [CH Tools]', 'Max Users [ClubHub]', 'Moderators [CH Tools]',
               'Moderators [New Members] [CH Tools]', 'Moderators [Old Members] [CH Tools]',
               'My Followers [ClubHub]', 'Speakers [CH Tools]', 'Speakers [New Members] [CH Tools]',
               'Speakers [Old Members] [CH Tools]', 'Total User Time [CH Tools]', 'Uniq Moderators [ClubHub]',
               'Uniq Speakers [ClubHub]', 'Uniq Users [ClubHub]']

columns_num = columns_pct + columns_int + columns_float

In [8]:
def column_data(columns=list, data=str):
    
    for c in columns:
        
        # set column dtypes
        if data == 'dt':
            df[c] = pd.to_datetime(df[c])
        
        elif data == 'cat':
            df[c] = df[c].astype('category')
            
        elif data == 'num':
            df[c] = df[c].astype(float)
#             df[c] = df[c].fillna(0)
    
#         remove these columns
#         elif data == 'drop':
#             df[c] = df.drop(columns=c)

In [9]:
column_data(columns_date, 'dt')
column_data(columns_cat, 'cat')
column_data(columns_num, 'num')

In [10]:
df = df[df['Room ID'].notna()]
df = df.drop(columns=columns_drop)

In [11]:
# df.sample(5)

In [12]:
# df.isna().sum()

In [13]:
# list(df.columns)

In [14]:
df['Uniq Listeners [ClubHub]'] = df['Uniq Users [ClubHub]'] - df['Uniq Speakers [ClubHub]']
df['Total Listener Time [ClubHub]'] = df['Uniq Listeners [ClubHub]'] * df['Avg Listener Minutes [ClubHub]'] 
df['Total Speaker Time [ClubHub]'] = df['Uniq Speakers [ClubHub]'] * df['Avg Speaker Minutes [ClubHub]']
df['Total User Time [ClubHub]'] = df['Total Speaker Time [ClubHub]'] + df['Total Listener Time [ClubHub]']
df['Avg Room Time [ClubHub]'] = df['Total User Time [ClubHub]'] / df['Uniq Users [ClubHub]']
df['New Users [ClubHub]'] = (df['Joined [In Last 7 Days] [ClubHub]'] * df['Uniq Users [ClubHub]']).round()
df['Old Users [ClubHub]'] = ((df['Joined [In Last Month] [ClubHub]'] + df['Joined [Over A Month] [ClubHub]']) * df['Uniq Users [ClubHub]']).round()


df['Users [CH Tools]'] = df['Listeners [CH Tools]'] + df['Speakers [CH Tools]']                               
df['New Users [CH Tools]'] = df['Listeners [New Members] [CH Tools]'] + df['Speakers [New Members] [CH Tools]']
df['Old Users [CH Tools]'] = df['Listeners [Old Members] [CH Tools]'] + df['Speakers [Old Members] [CH Tools]']


df['Duration'] = df[['Duration [CH Tools]', 'Duration [ClubHub]']].max(axis=1)
df['Listeners'] = df[['Listeners [CH Tools]', 'Uniq Listeners [ClubHub]']].max(axis=1)
df['Speakers'] = df[['Speakers [CH Tools]', 'Uniq Speakers [ClubHub]']].max(axis=1)
df['Moderators'] = df[['Moderators [CH Tools]', 'Uniq Moderators [ClubHub]']].max(axis=1)
df['Followers'] = df[['Followers [CH Tools]', 'My Followers [ClubHub]']].max(axis=1)
df['Users'] = df[['Users [CH Tools]', 'Uniq Users [ClubHub]']].max(axis=1)
df['Total User Time'] = df[['Total User Time [CH Tools]', 'Total User Time [ClubHub]']].max(axis=1)
df['Avg User Time'] = df[['Avg User Time [CH Tools]', 'Avg Room Time [ClubHub]']].max(axis=1)
df['New Users'] = df[['New Users [CH Tools]', 'New Users [ClubHub]']].max(axis=1)
df['Old Users'] = df[['Old Users [CH Tools]', 'Old Users [ClubHub]']].max(axis=1)
df['Stickiness [ClubHub]'] = 1 - df['Bounce [ClubHub]']

df['New Users Pct'] = (df['New Users'] / df['Users']).round(2)
df['Old Users Pct'] = (df['Old Users'] / df['Users']).round(2)

In [15]:
# df.sample(5)

In [16]:
# list(df.columns)

In [17]:
columns = ['Event', 'Users', 'New Users', 'Old Users', 'New Users Pct', 'Old Users Pct',
           'Users [CH Tools]', 'New Users [CH Tools]', 'Old Users [CH Tools]', 
           'Uniq Users [ClubHub]','New Users [ClubHub]', 'Old Users [ClubHub]']

# df[columns].sample(5)

In [18]:
columns = ['Room ID', 'Title', 'Event',
           'New Users', 'Old Users', 'Users', 
           'New Users Pct', 'Old Users Pct', 'Pct Sum',
           'New Users [CH Tools]', 'Old Users [CH Tools]', 'Users [CH Tools]',
           'New Users [ClubHub]', 'Old Users [ClubHub]', 'Uniq Users [ClubHub]']


# columns = ['Name', 'Event', 'Users', 'New Users', 'Old Users', 'New Users Pct', 'Old Users Pct',
#            'Pct Sum', 'Users [CH Tools]', 'New Users [CH Tools]', 'Old Users [CH Tools]', 
#            'Uniq 'sers [ClubHub]','New Users [ClubHub]', 'Old Users [ClubHub]']


df_check = df.copy()
df_check['Pct Sum'] = df['New Users Pct'] + df['Old Users Pct']
df_check = df_check.loc[df_check['Pct Sum'] > 1]
df_check[columns];

In [19]:
df['Users2'] = df['New Users'] + df['Old Users']
df['Users Diff'] = abs(df['Users'] - df['Users2'])
df['Users Pct'] = df['Users Diff'] / df['Users'] * 100

In [20]:
columns = ['Room ID', 'Title', 'Event', 'Users Pct', 'Users Diff', 'Users', 'Users2']

df_check = df.copy()
df_check = df_check.loc[df_check['Users Pct'] > 10]
df_check[columns];

In [21]:
# list(df.columns)

In [22]:
# df.isna().sum()

In [23]:
# list(df.columns.sort_values())
# list(df.columns)

In [24]:
columns = ['Room ID', 'Title', 'Event', 'Club', 'Tag 1', 'Tag 2', 'Duration', 'Users', 'Max Users [ClubHub]',
           'Moderators', 'Speakers', 'Listeners', 'Bounce [ClubHub]', 'Stickiness [ClubHub]', 'Total User Time', 
           'Avg User Time', 'Avg Listened Minutes [New Members] [CH Tools]', 'Avg Listened Minutes [Old Members] [CH Tools]',
           'Avg Speaker Minutes [ClubHub]', 'Avg Listener Minutes [ClubHub]', 'New Users', 
           'New Users Pct', 'Old Users', 'Old Users Pct', 'Followers', 'URL [ClubHub]', 'URL [CH Tools]', 'URL 2 [CH Tools]']

df = df[columns].copy()

In [25]:
df.rename(
    {'Room ID': 'Room_ID', 
     'Tag 1': 'Tag_1', 
     'Tag 2': 'Tag_2', 
     'Max Users [ClubHub]': 'Max_Room_Size',
     'Bounce [ClubHub]': 'Bounce', 
     'Stickiness [ClubHub]': 'Stickiness', 
     'Total User Time': 'Total_User_Time', 
     'Avg User Time': 'Avg_User_Time', 
     'Avg Listened Minutes [New Members] [CH Tools]': 'Avg_User_Time_New_Members', 
     'Avg Listened Minutes [Old Members] [CH Tools]': 'Avg_User_Time_Old_Members',
     'Avg Speaker Minutes [ClubHub]': 'Avg_Speaker_Time', 
     'Avg Listener Minutes [ClubHub]': 'Avg_Listener_Time', 
     'New Users': 'New_Users',
     'New Users Pct': 'New_Users_Pct', 
     'Old Users': 'Old_Users', 
     'Old Users Pct': 'Old_Users_Pct',
     'URL [ClubHub]': 'URL_ClubHub',
     'URL [CH Tools]': 'URL_CH_Tools_1', 
     'URL 2 [CH Tools]': 'URL_CH_Tools_2'}, 
    axis=1, inplace=True)

In [26]:
# df.rename(
#     {'Max Users [ClubHub]': 'Max Room Size', 
#      'Bounce [ClubHub]': 'Bounce',
#      'Stickiness [ClubHub]': 'Stickiness',
#      'Avg Listened Minutes [New Members] [CH Tools]': 'Avg User Time [New Members]', 
#      'Avg Listened Minutes [Old Members] [CH Tools]': 'Avg User Time [Old Members]',
#      'Avg Speaker Minutes [ClubHub]': 'Avg Speaker Time',
#      'Avg Listener Minutes [ClubHub]': 'Avg Listener Time'}, 
#     axis=1, inplace=True)

In [27]:
# df.sample(5)

In [28]:
# df.describe().transpose()

In [29]:
# df.columns

In [30]:
drop = ['Tag_1', 'Tag_2', 'URL_CH_Tools_1', 'URL_CH_Tools_2', 'URL_ClubHub']

df_save = df.copy()
df_save = df_save.drop(columns=drop)

In [31]:
df_save.to_csv('ch_analytics_clean.csv')
print('Process Complete')

Process Complete
